In [44]:
## import libraries
import pandas as pd
import numpy as np
import re
import difflib
import swifter

### WO file correction -- 2023

In [45]:
## correction of WO/ reactive file 2023

dfr = pd.read_excel("/Users/yashvardhansingh/Desktop/Valiance/cocacola/phase_2/By Dispenser Aggs/2023 reactive service FS9X.xlsx")
column_name_change = {'part_id':'date','platform2':'Platform','priloc':'Issue_desc','Serial Number':'serial_num','customer_name__c':'Chain_Name__c'}
dfr = dfr.rename(columns=column_name_change)
dfi = pd.read_excel("/Users/yashvardhansingh/Desktop/Valiance/cocacola/phase_2/DECAF Install Base v3.xlsx")
column_name_change = {'acn':'outletacn','FS_Serial_Number__c':'serial_num'}
dfi = dfi.rename(columns=column_name_change)

In [46]:
## variables
date = 'date'
wo_num = 'workordernumber'
outlet_num = 'outletacn'
platform = 'Platform'
cost = 'invoice_total__c'
issue = 'Issue_desc'
serial_num = 'serial_num'
city = 'Outlet_City__c'
state = 'Outlet_State_Province__c'
country = 'Outlet_Country__c'
chain_name = 'Chain_Name__c'
channel = 'trade_channel'
install_date = 'DECAF Install Date'
removal_date = 'DECAF Removal Date'

In [47]:
level = [date,wo_num,platform,outlet_num,serial_num,cost,issue]
dfr = dfr[level].reset_index(drop=True)
dfi = dfi[~dfi[serial_num].isna()].reset_index(drop=True)

In [50]:
## percentage of serial number found in install base
found = dfr[dfr[serial_num].isin(dfi[serial_num].unique())][serial_num].nunique()
total = dfr[serial_num].nunique()
print('Correct serial in unique terms %: {}'.format(round(found/total,2)))

Correct serial in unique terms %: 0.81


In [49]:
## do basic correction in serial numbers
dfr[serial_num] = dfr[serial_num].apply(lambda x: re.sub(r'\W+', '', x).upper()if pd.notna(x) else x)

In [51]:
## date correction
dfr[date] = pd.to_datetime(dfr[date], format='%Y%m%d')
fill_date = pd.to_datetime('2150-1-1')
dfi[removal_date].fillna(fill_date,inplace=True)

In [52]:
def close_match(df,dfi = dfi):
    serial = df[serial_num].iloc[0]
    outlet_no = df[outlet_num].iloc[0]
    platform_no = df[platform].iloc[0]
    date_no = df[date].iloc[0]
    new_serials = dfi[(dfi[outlet_num]==outlet_no)&(dfi[platform]==platform_no)&(dfi[install_date]<date_no)&(dfi[removal_date]>date_no)][serial_num].unique().tolist()
    approx_serial = dfi[(dfi[serial_num]==serial)&(dfi[install_date]<date_no)&(dfi[removal_date]>date_no)][serial_num].unique().tolist()

    ## take care of cases where serial number is correct but outlet or platform is not
    if pd.isna(serial) and not new_serials:
        df['closest_match'] = None
        df['reason'] = 'no valid serial_num in both the sheets'
        return df
    elif not new_serials:
        if not pd.isna(serial) and approx_serial:
            df['closest_match'] = approx_serial[0]
            df['reason'] = 'Found exact match'
            return df
        else:
            df['closest_match'] = None
            df['reason'] = 'no valid serial_num in the Base Sheet'
            return df
    elif pd.isna(serial) and len(new_serials)>=1:
        df['closest_match'] = new_serials[0]
        df['reason'] = 'no valid serial_num in the WO Sheet but found in Base Sheet'
        return df
    elif not pd.isna(serial) and  len(new_serials)==1:
        if serial == new_serials[0]:
            df['closest_match'] = new_serials[0]
            df['reason'] = 'Found exact match'
            return df
        else:
            df['closest_match'] = new_serials[0]
            df['reason'] = 'Not exact match but only one in Base Sheet'
            return df
    elif not pd.isna(serial) and  len(new_serials)>1:
        if serial in new_serials:
            df['closest_match'] = serial
            df['reason'] = 'Found exact match'
            return df
        elif approx_serial:
            df['closest_match'] = serial
            df['reason'] = 'Found exact match'
            return df
        else:
            matches = difflib.get_close_matches(serial, new_serials)
            if matches:
                df['closest_match'] = matches[0]
                df['reason'] = 'Found a Close Match from many serial nos in Base Sheet'
                return df
            else:
                df['closest_match'] = new_serials[0]
                df['reason'] = 'Found a random Match from many serial nos in Base Sheet'
                return df
    else:
        df['closest_match'] = None
        df['reason'] = 'No Usecase'
        return df

In [53]:
results = dfr.groupby([wo_num],dropna=False).apply(close_match).reset_index(drop=True)

In [54]:
results.to_excel("WO_sheet_modified_2023.xlsx",index=False)

### WO file correction -- 2022

In [35]:
## correction of WO/ reactive file 2023

dfr = pd.read_excel("/Users/yashvardhansingh/Desktop/Valiance/cocacola/phase_2/2022 - By Dispenser aggs/2022 Reactive service - By Dispenser.xlsx")
column_name_change = {'part_id':'date','platform2':'Platform','priloc':'Issue_desc','Serial Number':'serial_num','customer_name__c':'Chain_Name__c'}
dfr = dfr.rename(columns=column_name_change)
dfi = pd.read_excel("/Users/yashvardhansingh/Desktop/Valiance/cocacola/phase_2/DECAF Install Base v3.xlsx")
column_name_change = {'acn':'outletacn','FS_Serial_Number__c':'serial_num'}
dfi = dfi.rename(columns=column_name_change)

In [36]:
## variables
date = 'date'
wo_num = 'workordernumber'
outlet_num = 'outletacn'
platform = 'Platform'
cost = 'invoice_total__c'
issue = 'Issue_desc'
serial_num = 'serial_num'
city = 'Outlet_City__c'
state = 'Outlet_State_Province__c'
country = 'Outlet_Country__c'
chain_name = 'Chain_Name__c'
channel = 'trade_channel'
install_date = 'DECAF Install Date'
removal_date = 'DECAF Removal Date'

In [37]:
level = [date,wo_num,platform,outlet_num,serial_num,cost,issue]
dfr = dfr[level].reset_index(drop=True)
dfr[serial_num] = dfr[serial_num].astype(str)
dfi = dfi[~dfi[serial_num].isna()].reset_index(drop=True)

In [38]:
## percentage of serial number found in install base
found = dfr[dfr[serial_num].isin(dfi[serial_num].unique())][serial_num].nunique()
total = dfr[serial_num].nunique()
print('Correct serial in unique terms %: {}'.format(round(found/total,2)))

Correct serial in unique terms %: 0.59


- initial -- 0.59
- after standardization --- 0.77

In [39]:
## do basic correction in serial numbers
dfr[serial_num] = dfr[serial_num].apply(lambda x: re.sub(r'\W+', '', x).upper()if pd.notna(x) else x)

In [40]:
## date correction
dfr[date] = pd.to_datetime(dfr[date], format='%Y%m%d')
fill_date = pd.to_datetime('2150-1-1')
dfi[removal_date].fillna(fill_date,inplace=True)

In [41]:
def close_match(df,dfi = dfi):
    serial = df[serial_num].iloc[0]
    outlet_no = df[outlet_num].iloc[0]
    platform_no = df[platform].iloc[0]
    date_no = df[date].iloc[0]
    new_serials = dfi[(dfi[outlet_num]==outlet_no)&(dfi[platform]==platform_no)&(dfi[install_date]<date_no)&(dfi[removal_date]>date_no)][serial_num].unique().tolist()
    approx_serial = dfi[(dfi[serial_num]==serial)&(dfi[install_date]<date_no)&(dfi[removal_date]>date_no)][serial_num].unique().tolist()

    ## take care of cases where serial number is correct but outlet or platform is not
    if pd.isna(serial) and not new_serials:
        df['closest_match'] = None
        df['reason'] = 'no valid serial_num in both the sheets'
        return df
    elif not new_serials:
        if not pd.isna(serial) and approx_serial:
            df['closest_match'] = approx_serial[0]
            df['reason'] = 'Found exact match'
            return df
        else:
            df['closest_match'] = None
            df['reason'] = 'no valid serial_num in the Base Sheet'
            return df
    elif pd.isna(serial) and len(new_serials)>=1:
        df['closest_match'] = new_serials[0]
        df['reason'] = 'no valid serial_num in the WO Sheet but found in Base Sheet'
        return df
    elif not pd.isna(serial) and  len(new_serials)==1:
        if serial == new_serials[0]:
            df['closest_match'] = new_serials[0]
            df['reason'] = 'Found exact match'
            return df
        else:
            df['closest_match'] = new_serials[0]
            df['reason'] = 'Not exact match but only one in Base Sheet'
            return df
    elif not pd.isna(serial) and  len(new_serials)>1:
        if serial in new_serials:
            df['closest_match'] = serial
            df['reason'] = 'Found exact match'
            return df
        elif approx_serial:
            df['closest_match'] = serial
            df['reason'] = 'Found exact match'
            return df
        else:
            matches = difflib.get_close_matches(serial, new_serials)
            if matches:
                df['closest_match'] = matches[0]
                df['reason'] = 'Found a Close Match from many serial nos in Base Sheet'
                return df
            else:
                df['closest_match'] = new_serials[0]
                df['reason'] = 'Found a random Match from many serial nos in Base Sheet'
                return df
    else:
        df['closest_match'] = None
        df['reason'] = 'No Usecase'
        return df

In [42]:
results = dfr.groupby([wo_num],dropna=False).apply(close_match).reset_index(drop=True)

In [43]:
results.to_excel("WO_sheet_modified_2022.xlsx",index=False)

### MODEL - 1